In [1]:
import pandas as pd
import numpy as np
from pandas.core.dtypes.common import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from IPython.display import display
from sklearn import metrics
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, ParameterGrid
from scipy.stats import randint as sp_randint
import statsmodels.api as sm
from sklearn.metrics import classification_report,confusion_matrix
%matplotlib inline

/anaconda/envs/fastai/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Steps for logistic regression 
 1.Look at distribution of response variable to determine if it's a balanced, imbalanced or multi-class problem.<br>
 2.Finalise metric <br>
 3.Find the numerical features and scale them <br>
 4.Determine which columns are categorical(all columns except target variable) and one hot encode them <br>
 5.Select a splitting method(random, last) and split it into train and validation sets. <br>
 6.Check if cross fold validation is a good idea <br>
 7.Hyper-parameter optimisation <br>
 8.Train final model on complete data <br>

In [2]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)

def train_cats(df):
    for col_name, col in df.items():
        if is_string_dtype(col): df[col_name] = col.astype('category').cat.as_ordered()

def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum():
            df[name] = col.fillna(col.median())
            
# Convert string columns in test df into categorical columns, using cat codes from training data
def apply_cats(df, trn):
    for n,c in df.items():
        if (n in trn.columns) and (trn[n].dtype.name=='category'):
            df[n] = pd.Categorical(c, categories=trn[n].cat.categories, ordered=True)
            
def scale_vars(df, mapper):
#     warnings.filterwarnings('ignore', category=sklearn.exceptions.DataConversionWarning)
    if mapper is None:
        map_f = [([n],StandardScaler()) for n in df.columns if is_numeric_dtype(df[n])]
        mapper = DataFrameMapper(map_f).fit(df)
    df[mapper.transformed_names_] = mapper.transform(df)
    return mapper

def numericalize(df, col, name, max_n_cat):
    if not is_numeric_dtype(col) and ( max_n_cat is None or col.nunique()>max_n_cat):
        df[name] = col.cat.codes+1
        
def proc_df(df, y_fld, do_scale=False, max_n_cat=None, mapper=None,nonlinear=True):
    df = df.copy()
    y = df[y_fld].values
    df.drop([y_fld], axis=1, inplace=True)
    for n,c in df.items(): fix_missing(df, c, n)
    if do_scale: mapper = scale_vars(df, mapper)
    if nonlinear:
        for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    res = [pd.get_dummies(df, dummy_na=True), y]
    if do_scale: res = res + [mapper]
    return res
def score_for_fi(x1, x2):
    # this controls the metric used for feature importance calculation
    return metrics.mean_squared_error(x1, x2)



In [3]:
dta = sm.datasets.fair.load_pandas().data
# add "affair" column: 1 represents having affairs, 0 represents not
dta['affair'] = (dta.affairs > 0).astype(int)
dta.drop('affairs',axis=1,inplace=True)

In [4]:
dta.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,1
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,1
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,1
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,1


Dataset
The dataset is the affairs dataset that comes with Statsmodels. It was derived from a survey of women in 1974 by Redbook magazine, in which married women were asked about their participation in extramarital affairs. More information about the study is available in a 1978 paper from the Journal of Political Economy.

Description of Variables
The dataset contains 6366 observations of 9 variables:

rate_marriage: woman's rating of her marriage (1 = very poor, 5 = very good)
age: woman's age
yrs_married: number of years married
children: number of children
religious: woman's rating of how religious she is (1 = not religious, 4 = strongly religious)
educ: level of education (9 = grade school, 12 = high school, 14 = some college, 16 = college graduate, 17 = some graduate school, 20 = advanced degree)
occupation: woman's occupation (1 = student, 2 = farming/semi-skilled/unskilled, 3 = "white collar", 4 = teacher/nurse/writer/technician/skilled, 5 = managerial/business, 6 = professional with advanced degree)
occupation_husb: husband's occupation (same coding as above)
affairs: time spent in extra-marital affairs

In [5]:
# Number of unique elements in each column

In [6]:
dta.T.apply(lambda x: x.nunique(),axis=1)

rate_marriage      5
age                6
yrs_married        7
children           6
religious          4
educ               6
occupation         6
occupation_husb    6
affair             2
dtype: int64

In [7]:
dta.iloc[:,0:7]=dta.iloc[:,0:7].apply(lambda x:x.astype('category'),axis=1)

In [8]:
for i in range(0,8):
    col = dta.columns[i]
    dta[col]=dta[col].astype('category')

In [9]:

dta.dtypes

rate_marriage      category
age                category
yrs_married        category
children           category
religious          category
educ               category
occupation         category
occupation_husb    category
affair                int64
dtype: object

In [10]:
X,y= proc_df(dta,'affair',nonlinear=False)

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25,random_state=0)

LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)

Instructions:
penalty='l2' 
Use dual: only when no of features> no of samples
C=1.0 Inverse of lambda ( regularisation factor)
multi_class='ovr' This is the standard technique ( one over rest, other options include multinomial: could be used with many solvers apart from the default solver 'liblinear' for faster convergence

In [12]:
lm = LogisticRegression(C=0.5,class_weight='balanced',
                        multi_class='multinomial',solver='lbfgs',warm_start=True,n_jobs=-1, max_iter= 200)
lm.fit(X, y)
lm.score(X_train,y_train)

0.6897779639715124

In [13]:
lm.score(X_valid,y_valid)

0.6991206030150754

In [17]:
print(metrics.confusion_matrix(y_valid, lm.predict(X_valid)))
print(metrics.classification_report(y_valid, lm.predict(X_valid)))

[[779 321]
 [158 334]]
             precision    recall  f1-score   support

          0       0.83      0.71      0.76      1100
          1       0.51      0.68      0.58       492

avg / total       0.73      0.70      0.71      1592



No good summary available in sklearn, we might need to switch to statsmodel for the same. But I dont think we need to
more than what is written above

# Adhoc Analysis

In [16]:
data2 = sm.datasets.ccard.load_pandas().data

In [17]:
data2.head()

,AVGEXP,AGE,INCOME,INCOMESQ,OWNRENT
0,124.98,38.0,4.52,20.4304,1.0
1,9.85,33.0,2.42,5.8564,0.0
2,15.00,34.0,4.50,20.2500,1.0
3,137.87,31.0,2.54,6.4516,0.0
4,546.50,32.0,9.79,95.8441,1.0


In [18]:
data2.dtypes

AVGEXP      float64
AGE         float64
INCOME      float64
INCOMESQ    float64
OWNRENT     float64
dtype: object

In [19]:
for col in ['AGE','INCOMESQ']:
    data2[col] = data2[col].astype('category')

This dataset has both categorical and numerical variables


In [20]:
data2.dtypes

AVGEXP       float64
AGE         category
INCOME       float64
INCOMESQ    category
OWNRENT      float64
dtype: object

In [29]:
data2.shape

(72, 5)

In [21]:
X,y,_= proc_df(data2,'OWNRENT',nonlinear=False,do_scale=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [22]:
lm = LogisticRegression(C=0.6,class_weight='balanced',multi_class='multinomial',
                        solver='lbfgs',warm_start='True', max_iter=200)
lm.fit(X_train, y_train)
lm.score(X_train,y_train)

0.9629629629629629

In [23]:
lm.score(X_test,y_test)

0.7777777777777778

# Hyper parameter tuning

## 6.Logistic regression

In [19]:
from scipy import stats as st
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, ParameterGrid
import warnings
warnings.filterwarnings("ignore")

Clist = list(st.expon(0).rvs(size=20))
grid = {
    'C': Clist,
    'max_iter':[200],
}
logit_search = LogisticRegression(C=0.5,class_weight='balanced',n_jobs=-1, max_iter= 200)
m_cv = RandomizedSearchCV(estimator=logit_search, param_distributions=grid,verbose=1)
m_cv.fit(X_train, y_train)
print("Best parameters are: {}".format(m_cv.best_params_))
print("Best score is {}".format(m_cv.best_score_))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters are: {'max_iter': 200, 'C': 4.091407244026523}
Best score is 0.6908253037285296


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.3s finished


In [21]:
logit_final = m_cv.best_estimator_
logit_final.fit(X_train, y_train)
logit_r2 = logit_final.score(X_valid, y_valid)

## 7.Random forest classifier

In [23]:
from hpsklearn import HyperoptEstimator,random_forest
from hyperopt import tpe
from sklearn.metrics import log_loss
import numpy as np
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25,random_state=0)
X_train = X_train.values
X_valid = X_valid.values

estim = HyperoptEstimator(regressor=random_forest('affairs'),
                          preprocessing=[],
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)
estim.fit( X_train, y_train )
print(estim.score( X_valid, y_valid))

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely
0.7273869346733668


In [25]:
final_rf =estim.best_model()['learner']
final_rf.fit(X_train, y_train)
rf_r2 = final_rf.score(X_valid, y_valid)

## 8.Extra tree classifier

In [26]:
from hpsklearn import HyperoptEstimator,extra_trees
from hyperopt import tpe
from sklearn.metrics import log_loss
import numpy as np
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25,random_state=0)
X_train = X_train.values
X_valid = X_valid.values

estim = HyperoptEstimator(regressor=extra_trees('affairs'),
                          preprocessing=[],
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)
estim.fit( X_train, y_train )
print(estim.score( X_valid, y_valid))

0.7248743718592965


In [27]:
final_et =estim.best_model()['learner']
final_et.fit(X_train, y_train)
et_r2 = final_et.score(X_valid, y_valid)

# Comparison of models post tuning

In [31]:
r2_score = [rf_r2,et_r2,logit_r2]
models = ['Random Forest','Extra Tree Classifier','Logistic Regression']
pd.DataFrame({'Models':models,'R2':r2_score}).sort_values(by='R2',ascending=False)

,Models,R2
0,Random Forest,0.727387
1,Extra Tree Classifier,0.724874
2,Logistic Regression,0.690955
